In [1]:
from braindecode.preprocessing import (
    exponential_moving_standardize, 
    preprocess, 
    Preprocessor)
from numpy import multiply
from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import create_windows_from_events
import torch
from braindecode.util import set_random_seeds

from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import pandas as pd
from sklearn.metrics import confusion_matrix
from braindecode.visualization import plot_confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
import mne
import os
import sys
from mne.datasets import eegbci
import glob
from IPython.display import clear_output
import numpy as np
import torch
from torch import nn
import torch.optim as optim

from mne.datasets import eegbci
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from scipy import signal


import torch
import torch.cuda as cuda
import torch.nn as nn
from torch.autograd import Variable
import math
import wandb
import os
import neptune.new.integrations.sklearn as npt_utils
import neptune.new as neptune

os.environ['http_proxy'] = "http://192.41.170.23:3128"
os.environ['https_proxy'] = "http://192.41.170.23:3128"

from braindecode.models import ShallowFBCSPNet, Deep4Net ,EEGNetv4


/home/nutapolt/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
def create_dataloader(X, y, batch_size):
    X_tensor = torch.tensor(X).float()
    y_tensor = torch.tensor(y).long()
    dataset_tensor = TensorDataset(X_tensor, y_tensor)
    dl = torch.utils.data.DataLoader(dataset_tensor, batch_size=batch_size, shuffle=True
                                     )
    return dl

def early_stopping(train_loss, validation_loss, min_delta, tolerance):
    counter = 0
    if (validation_loss - train_loss) > min_delta:
        counter +=1
        if counter >= tolerance:
          return True

def extrack_dataset(dataset):
    for x, y, window_ind in dataset:
        x_shape = x.shape
        y_shape = len(dataset.get_metadata().target)
        break
    X = np.zeros((y_shape,x_shape[0],x_shape[1]))
    y_=[]
    i=0
    for x, y, window_ind in dataset:
        X[i]=x
        y_.append(y)
        i+=1
    #X2 = X[:, 7:8, :]
    #X3= X[:, 11:12, :]
    #(288, 22, 1125)
    #X = np.concatenate((X2,X3), axis=1)
    print(X.shape)
    X = X.reshape(y_shape,X.shape[2],X.shape[1])
    #X = X.reshape(y_shape,X.shape[2],X.shape[1])
    return X,np.array(y_).T


def extrack_dataset_2class(dataset):
    for x, y, window_ind in dataset:
        x_shape = x.shape
        y_shape = len(dataset.get_metadata().target)
        break
    X = np.zeros((y_shape,x_shape[0],x_shape[1]))
    y_=[]
    i=0
    for x, y, window_ind in dataset:
        X[i]=x
        y_.append(y)
        i+=1
    y = np.array(y_).T
    X_ = np.zeros((int(y_shape/2) ,x_shape[0],x_shape[1])) #for 2 class
    y_for_2class =[]
    j=0
    for i in range(y.shape[0]):
        if y[i] == 0 or y[i] == 1:
            y_for_2class.append(y[i])
            X_[j]=X[i]
            j +=1
    y_for_2class = np.array(y_for_2class).T
    X_ = X_.reshape(y_for_2class.shape[0],X_.shape[2],X_.shape[1])
    return X_,y_for_2class

def extrack_dataset_2class_cut(dataset):
    for x, y, window_ind in dataset:
        x_shape = x.shape
        y_shape = len(dataset.get_metadata().target)
        break
    X = np.zeros((y_shape,x_shape[0],x_shape[1]))
    y_=[]
    i=0
    for x, y, window_ind in dataset:
        X[i]=x
        y_.append(y)
        i+=1
    y = np.array(y_).T
    X_ = np.zeros((int(y_shape/2) ,x_shape[0],x_shape[1])) #for 2 class
    y_for_2class =[]
    j=0
    for i in range(y.shape[0]):
        if y[i] == 0 or y[i] == 1:
            y_for_2class.append(y[i])
            X_[j]=X[i]
            j +=1
    y_for_2class = np.array(y_for_2class).T
    X_ = X_.reshape(y_for_2class.shape[0],X_.shape[2],X_.shape[1])
    #100, 1, 576, 22
    X_ = X_[:,64:320,:]
    print(X_.shape)
    return X_,y_for_2class


def train(model,gpu_num,train_loader,test_loader,
          weights_name=False,
          optimizer = None,
          criterion = None,
          num_epochs=500,
          vail_loader= None,
          save_weights = False,
          neptune = True,
          lr = None
         ):

    # Train the model


    train_loss = []
    valid_loss = [10,11]
    train_accuracy = []
    valid_accuracy = []

    old_loss = 100
    old_acc = 0
    valid_loss_vail = []


    for epoch in range(num_epochs):
        iter_loss = 0.0
        correct = 0
        iterations = 0

        model.train()

        for i, (items, classes) in enumerate(train_loader):
            items = Variable(items)
            classes = classes.type(torch.LongTensor)
            classes = Variable(classes)

            if cuda.is_available():
                items = items.cuda(gpu_num)
                classes = classes.cuda(gpu_num)

            optimizer.zero_grad()
            outputs = model(items)
            #avg_pedic = torch.mean(outputs, 1, True).float()
            #avg_pedic = avg_pedic.reshape(classes.shape[0])
            #print(avg_pedic.shape)
            #print(classes.shape)
            loss = criterion(outputs, classes)

            iter_loss += loss.item()
            loss.backward()
            optimizer.step()

            metrics = {"train/train_loss": loss}

            #print(loss)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == classes.data).sum()
            iterations += 1

        train_loss.append(iter_loss/iterations)


        train_accuracy.append(( correct.float() / len(train_loader.dataset)))
        train_metrics = {"train/train_loss": iter_loss/iterations,
                       "train/train_accuracy": (100 * correct.float() / len(train_loader.dataset))}



        loss = 0.0
        correct = 0
        iterations = 0
        model.eval()

        for i, (items, classes) in enumerate(test_loader):
            classes = classes.type(torch.LongTensor)
            items = Variable(items)
            classes = Variable(classes)

            if cuda.is_available():
                items = items.cuda(gpu_num)
                classes = classes.cuda(gpu_num)


            outputs = model(items)
            #avg_pedic = torch.mean(outputs, 1, True).float()
            #avg_pedic = avg_pedic.reshape(classes.shape[0])

            loss += criterion(outputs, classes).item()

            _, predicted = torch.max(outputs.data, 1)

            correct += (predicted == classes.data).sum()
            #print("correct : {}".format(classes.data))
            #print("predicted : {}".format(predicted))
            iterations += 1

        valid_loss.append(loss/iterations)
        correct_scalar = np.array([correct.clone().cpu()])[0]
        valid_accuracy.append(correct_scalar / len(test_loader.dataset) )

        test_metrics = {"Test/Test_loss": loss/iterations,
                       "Test/Test_accuracy": correct_scalar / len(test_loader.dataset) }
        if save_weights is True:
            if epoch+1 > 2 and valid_loss[-1] < old_loss and old_acc <= valid_accuracy[-1] :
                    newpath = r'./{}'.format(weights_name)
                    if not os.path.exists(newpath):
                        os.makedirs(newpath)
                    torch.save(model.state_dict(),'./{}/{:.4f}_{}_{:.4f}_{:.4f}'.format(weights_name,valid_loss[-1],weights_name,valid_loss[-1],valid_accuracy[-1]))
                    part_weights = './{}/{:.4f}_{}_{:.4f}_{:.4f}'.format(weights_name,valid_loss[-1],weights_name,valid_loss[-1],valid_accuracy[-1])
                    old_loss = valid_loss[-1]
                    old_acc = valid_accuracy[-1]

        print ('Epoch %d/%d, Tr Loss: %.4f, Tr Acc: %.4f, Val Loss: %.4f, Val Acc: %.4f , le : %f'
                       %(epoch+1, num_epochs, train_loss[-1], train_accuracy[-1], valid_loss[-1], valid_accuracy[-1],lr))
        if early_stopping(train_loss[-1], valid_loss[-1], min_delta=10, tolerance = 20):
            print("We are at epoch:", epoch+1)
            run[f"epoch/valid_accuracy"].append(valid_accuracy[-1])
            run[f"epoch/train_accuracy"].append(train_accuracy[-1])
            run[f"epoch/train_loss"].append(train_loss[-1])
            run[f"epoch/valid_loss"].append(valid_loss[-1])

            break
        if neptune is True:
            run[f"epoch/valid_accuracy"].append(valid_accuracy[-1])
            run[f"epoch/train_accuracy"].append(train_accuracy[-1])
            run[f"epoch/train_loss"].append(train_loss[-1])
            run[f"epoch/valid_loss"].append(valid_loss[-1])


    return train_loss,valid_loss,train_accuracy,valid_accuracy,part_weights
def eval(model,
         gpu_num,
          vail_loader= None,
         labels=None,
         ):
    model.eval()
    y_pred = []
    y_true = []
    correct=0
    for i, (items, classes) in enumerate(vail_loader):
        classes = classes.type(torch.LongTensor)
        items = Variable(items)
        classes = Variable(classes)
        if cuda.is_available():
            items = items.cuda(gpu_num)
            classes = classes.cuda(gpu_num)

        outputs = model(items)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.append(predicted.clone().cpu().numpy())
        y_true.append(classes.data.clone().cpu().numpy())
        correct += (predicted == classes.data).sum()
    correct_scalar = np.array([correct.clone().cpu()])[0]
    valid_accuracy.append(correct_scalar / len(vail_loader.dataset) )

    confusion_mat = confusion_matrix(np.array(y_true).T,np.array(y_pred).T )
    run[f"epoch/eval_ACC"].append(valid_accuracy[-1])
    run["confusion matrices subject_id : {0}".format(subject_id)].upload(plot_confusion_matrix(confusion_mat, class_names=labels,rotate_row_labels=0,rotate_col_labels=90,with_f1_score=True))
    print(confusion_mat)
    return y_pred,y_true,correct_scalar,valid_accuracy

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 1125

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 22), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)

        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)

        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))

        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints.
        self.fc1 = nn.Linear(560, 4)


    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)

        # FC Layer
        x = x.reshape(x.size(0), -1)
        x = F.sigmoid(self.fc1(x))
        return x


In [ ]:
subjects = list(range(1,10))

low_cut_hz = 8.  # low cut frequency for filtering
high_cut_hz = 35.  # high cut frequency for filtering
resample = 128
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000
# Factor to convert from V to uV
factor = 1e6
preprocessors = [
                    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
                    Preprocessor(lambda data: multiply(data, factor)),  # Convert from V to uV
                    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),
                    #Preprocessor('resample', sfreq=resample),
                    Preprocessor(exponential_moving_standardize,  # Exponential movin standardization
                                factor_new=factor_new,
                                 init_block_size=init_block_size)
                    ]
n_epochs = 5000
lr = 0.001

weight_decay = 0.5 * 0.001
task_list = ['left', 'right', 'foot', 'tongue']
#task_list = ['tongue']
network_list = ['ATCNet','EEGITNet','ShallowFBCSPNet','Deep4Net']

#network_list = ['EEGITNet']
percent_list = [18,36,54,72]
#percent_list = [54]

In [ ]:
for subject_id in subjects:
    for percent in percent_list:

        run = neptune.init_run(
        project="AitBrainLab/Synthetic",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhMTMyMzg0My02NzlhLTQ3N2ItYTdmMS0yNTcwNDBmM2QwM2QifQ==",
)



        dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids= [subject_id])
        preprocess(dataset, preprocessors)
        trial_start_offset_seconds = -0.5
        # Extract sampling frequency, check that they are same in all datasets
        sfreq = dataset.datasets[0].raw.info['sfreq']
        print(sfreq)
        assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
        # Calculate the trial start offset in samples.
        trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

        # Create windows using braindecode function for this. It needs parameters to define how
        # trials should be used.
        windows_dataset = create_windows_from_events(
            dataset,
            trial_start_offset_samples=trial_start_offset_samples,
            trial_stop_offset_samples=0,
            preload=True,
        )
        splitted = windows_dataset.split('session')
        train_set = splitted['session_T']
        valid_set = splitted['session_E']

        input_window_samples = train_set[0][0].shape[1]
        X_train,y_train = extrack_dataset(train_set)
        X_train = X_train[:, np.newaxis,:,:]

        X_valid,y_valid = extrack_dataset(valid_set)
        X_syntheic = np.load("Synthetic_data_shift/X_subject_{0}.npy".format(subject_id)) #np.load("Synthetic_data/X_subject_3_20_[0, 1].npy")#
        y_syntheic = np.load("Synthetic_data_shift/y_subject_{0}.npy".format(subject_id))#np.load("Synthetic_data/y_subject_3_20_[0, 1].npy")#t))#np.load("Synthetic_data/y_subject_3_20_[0, 1].npy")#
        #['left', 'right', 'foot', 'tongue']
        if len(task_list) == 4:
            for task in task_list :
                if task == "left":
                    #X = X.reshape(y_shape,X.shape[2],X.shape[1])
                    y1 = y_syntheic[np.random.randint(0,72,percent)]
                    X1 = X_syntheic[np.random.randint(0,72,percent)]
                    X1=X1.reshape(y1.shape[0],X1.shape[2],X1.shape[1])
                if task == "right":
                    y2 = y_syntheic[np.random.randint(72,144,percent)]
                    X2 = X_syntheic[np.random.randint(72,144,percent)]
                    X2=X2.reshape(y2.shape[0],X2.shape[2],X2.shape[1])
                if task == "foot":
                    y3 = y_syntheic[np.random.randint(144,216,percent)]
                    X3 = X_syntheic[np.random.randint(144,216,percent)]
                    X3=X3.reshape(y3.shape[0],X3.shape[2],X3.shape[1])
                if task == "tongue":
                    y4 = y_syntheic[np.random.randint(216,288,percent)]
                    X4 = X_syntheic[np.random.randint(216,288,percent)]
                    X4=X4.reshape(y4.shape[0],X4.shape[2],X4.shape[1])
            X_syntheic = np.concatenate((X1,X2,X3,X4),axis=0)
            y_syntheic = np.concatenate((y1,y2,y3,y4),axis=0)
            X_syntheic = X_syntheic[:, np.newaxis,:,:]
        if len(task_list)==1:
            for task in task_list :
                if task == "left":
                    X_syntheic = X_syntheic[np.random.randint(0,72,percent)]
                    y_syntheic = y_syntheic[np.random.randint(0,72,percent)]
                if task == "right":
                    X_syntheic = X_syntheic[np.random.randint(72,144,percent)]
                    y_syntheic = y_syntheic[np.random.randint(72,144,percent)]
                if task == "foot":
                    X_syntheic = X_syntheic[np.random.randint(144,216,percent)]
                    y_syntheic = y_syntheic[np.random.randint(144,216,percent)]
                if task == "tongue":
                    X_syntheic = X_syntheic[np.random.randint(216,288,percent)]
                    y_syntheic = y_syntheic[np.random.randint(216,288,percent)]
            X_syntheic=X_syntheic.reshape(y_syntheic.shape[0],X_syntheic.shape[2],X_syntheic.shape[1])
            X_syntheic = X_syntheic[:, np.newaxis,:,:]
            X_train = np.concatenate((X_train,X_syntheic),axis = 0)
            y_train = np.concatenate((y_train, y_syntheic),axis = 0)




        X_valid = X_valid[:, np.newaxis,:,:]
        X_train, X_test, y_train, y_test = train_test_split(X_train,y_train, test_size=0.3,stratify=y_train)
        label_dict = valid_set.datasets[0].windows.event_id.items()
        labels =list(dict(sorted(list(label_dict), key=lambda kv: kv[1])).keys()) #['left_hand', 'right_hand']#list(dict(sorted(list(label_dict), key=lambda kv: kv[1])).keys())
        print('train size',X_train.shape, y_train.shape)
        print('test size',X_test.shape, y_test.shape)

        batch_size = X_train.shape[2]

        train_loader = create_dataloader(X_train, y_train, batch_size=batch_size)
        test_loader = create_dataloader(X_test, y_test, batch_size=batch_size)
        valid_loader = create_dataloader(X_valid, y_valid, batch_size=batch_size)
        n_classes=4
        n_chans = X_train.shape[1]


        input_window_samples = train_set[0][0].shape[1]
        model = EEGNet()
                #EEGNetv4(
                 #   n_chans,
                  #  n_classes,
                   # final_conv_length='auto',
                   # input_window_samples=input_window_samples,
                   # )





        params = {"Subject number":subject_id,
                      "learning_rate": lr ,
                      "optimizer": "AdamW" ,
                      "Network": "EEGNetv4",
                      "Datasets":"BNCI2014001+Synthetic_size_{0}.".format(percent),
                      "sfreq":dataset.datasets[0].raw.info['sfreq'],
                      "Class number":n_classes,
                      "Channel number": train_set[0][0].shape[0],
                      "samples point" : X_train.shape[2]

                      }
        run["parameters"] = params

        net = model.cuda(0)
        optimizer = optim.Adam(net.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()#nn.BCELoss()#
        train_loss,valid_loss,train_accuracy,valid_accuracy,part_weights =train(
                                                                    model = net,
                                                                    gpu_num = 0,
                                                                    train_loader = train_loader,
                                                                    test_loader = test_loader,
                                                                    optimizer = optimizer  ,
                                                                    criterion = criterion,
                                                                    num_epochs=n_epochs,
                                                                    save_weights= True,
                                                                    lr=lr
                                                                         )
        model.load_state_dict(torch.load(part_weights))
        eval(model = net,
            gpu_num = 0,
              vail_loader= valid_loader,
             labels=labels,
             )


        run.stop()



[[35 13 14 10]
 [ 2 54 13  3]
 [15 15 31 11]
 [27  5  4 36]]


/home/nutapolt/.local/lib/python3.8/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


([array([0, 2, 3, 1, 1, 3, 0, 1, 3, 1, 2, 2, 2, 3, 2, 0, 1, 0, 3, 1, 0, 0,
         3, 0, 1, 3, 1, 2, 1, 3, 1, 1, 3, 0, 2, 0, 1, 1, 2, 2, 2, 1, 0, 1,
         3, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 3, 1, 1, 0, 1, 1, 3, 1, 0, 2,
         0, 1, 0, 3, 2, 2, 0, 0, 0, 0, 2, 0, 1, 2, 1, 2, 0, 1, 2, 0, 3, 0,
         3, 2, 0, 2, 0, 2, 0, 3, 2, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1,
         1, 1, 2, 0, 3, 3, 2, 3, 1, 2, 2, 0, 0, 0, 1, 1, 3, 3, 0, 2, 3, 1,
         1, 3, 0, 0, 1, 2, 0, 1, 0, 3, 2, 3, 3, 0, 3, 2, 0, 3, 3, 0, 0, 2,
         1, 1, 3, 1, 1, 2, 1, 1, 1, 2, 2, 0, 0, 1, 1, 3, 3, 3, 3, 1, 3, 3,
         0, 2, 2, 3, 1, 3, 1, 3, 3, 1, 0, 2, 3, 3, 2, 0, 1, 1, 0, 3, 2, 1,
         0, 1, 3, 0, 0, 0, 1, 0, 0, 1, 2, 2, 1, 1, 0, 3, 1, 0, 2, 2, 0, 2,
         0, 3, 1, 2, 2, 3, 2, 3, 1, 1, 2, 1, 2, 0, 3, 0, 3, 2, 1, 1, 2, 0,
         0, 1, 1, 1, 3, 0, 3, 1, 2, 3, 1, 2, 3, 1, 1, 1, 2, 0, 0, 2, 2, 0,
         2, 0, 3, 1, 0, 2, 1, 3, 1, 3, 0, 3, 0, 1, 0, 3, 0, 0, 2, 1, 3, 0,
         1, 1])],
 [array